In [3]:
import pandas as pd 

play_by_play_condensed = pd.read_parquet("/Users/shaanchanchani/dev/nfl-data-api/cache/play_by_play_condensed.parquet")

play_by_play_condensed.head()

,play_id,game_id,old_game_id,season,week,posteam,defteam,side_of_field,passer_player_id,rusher_player_id,...,pass_oe,qb_epa,xyac_epa,xyac_mean_yardage,xyac_success,series_success,success,drive,fixed_drive_result,receiving_yards
0,1.0,2020_01_ARI_SF,2020091311,2020,1,None,None,None,None,None,...,NaN,0.000000,NaN,NaN,NaN,1.0,0.0,NaN,Field goal,NaN
1,39.0,2020_01_ARI_SF,2020091311,2020,1,SF,ARI,ARI,None,None,...,NaN,0.000000,NaN,NaN,NaN,1.0,0.0,1.0,Field goal,NaN
2,54.0,2020_01_ARI_SF,2020091311,2020,1,SF,ARI,SF,00-0031345,None,...,48.494154,1.294838,0.50337,4.275047,0.619306,1.0,1.0,1.0,Field goal,5.0
3,93.0,2020_01_ARI_SF,2020091311,2020,1,SF,ARI,SF,None,00-0031687,...,-41.335732,0.857214,NaN,NaN,NaN,1.0,1.0,1.0,Field goal,NaN
4,118.0,2020_01_ARI_SF,2020091311,2020,1,SF,ARI,ARI,None,00-0031687,...,-44.692025,-0.454665,NaN,NaN,NaN,0.0,0.0,1.0,Field goal,NaN


In [6]:
players_condensed = pd.read_parquet("/Users/shaanchanchani/dev/nfl-data-api/cache/players_condensed.parquet")

players_condensed.columns

write me a 


Index(['college_conference', 'current_team_id', 'display_name', 'draft_club',
       'draft_number', 'draftround', 'entry_year', 'esb_id', 'first_name',
       'football_name', 'gsis_id', 'gsis_it_id', 'jersey_number', 'last_name',
       'position', 'position_group', 'rookie_year', 'short_name', 'status',
       'status_description_abbr', 'status_short_description', 'team_abbr',
       'uniform_number', 'height', 'weight', 'headshot', 'smart_id',
       'college_name', 'years_of_experience', 'birth_date', 'team_seq',
       'suffix'],
      dtype='object')

In [8]:
import pandas as pd
from typing import Dict, List, Optional, Tuple
from datetime import datetime


def resolve_player(name: str, season: Optional[int] = None) -> Tuple[Optional[Dict], List[Dict]]:
    """Resolve player name to a single player or return alternatives.
    
    Args:
        name: Player name to resolve
        season: Optional season to use for resolution (defaults to current season)
        
    Returns:
        Tuple of (resolved player dict or None, list of alternative matches)
    """
    if not name:
        return None, []
    
    if season is None:
        season = get_current_season()
        
    # Load both current players and historical roster data
    players_df = await load_players()
    try:
        roster_df = await load_rosters([season])
        # Merge roster data with player data to get historical team info
        if not roster_df.empty:
            players_df = players_df.merge(
                roster_df[['gsis_id', 'team', 'season']],
                left_on='gsis_id',
                right_on='gsis_id',
                how='left'
            )
            # Update team_abbr with historical team info where available
            players_df.loc[players_df['season'] == season, 'team_abbr'] = players_df.loc[players_df['season'] == season, 'team']
    except Exception:
        # If roster data isn't available, continue with current data
        pass
        
    name_lower = name.lower()
    
    # Try exact match on display_name first
    exact_matches = players_df[players_df['display_name'].str.lower() == name_lower]
    if len(exact_matches) == 1:
        return exact_matches.iloc[0].to_dict(), []
    
    # Try matching both name formats (First Last and Last, First)
    if ',' in name:
        # Convert "Last, First" to "First Last"
        last, first = name_lower.split(',')
        alt_name = f"{first.strip()} {last.strip()}"
    else:
        # Convert "First Last" to "Last, First"
        parts = name_lower.split()
        if len(parts) >= 2:
            alt_name = f"{parts[-1]}, {' '.join(parts[:-1])}"
        else:
            alt_name = name_lower
            
    name_matches = players_df[
        (players_df['display_name'].str.lower() == name_lower) |
        (players_df['display_name'].str.lower() == alt_name)
    ]
    if len(name_matches) == 1:
        return name_matches.iloc[0].to_dict(), []
    if len(name_matches) > 1:
        return None, name_matches.to_dict('records')
    
    # Try contains match as fallback
    contains_matches = players_df[
        players_df['display_name'].str.lower().str.contains(name_lower, na=False) |
        players_df['display_name'].str.lower().str.contains(alt_name, na=False)
    ]
    if len(contains_matches) == 1:
        return contains_matches.iloc[0].to_dict(), []
    elif len(contains_matches) > 1:
        return None, contains_matches.to_dict('records')
    
    return None, []


SyntaxError: 'await' outside async function (790629033.py, line 23)

In [7]:
players_condensed['gsis_id']

0        00-0004866
1        00-0032889
2        00-0037845
3        00-0039793
4        00-0030228
            ...    
20753    00-0039689
20754    00-0022024
20755    00-0030855
20756    00-0037373
20757    00-0037268
Name: gsis_id, Length: 20758, dtype: object